In [2]:
from google.cloud import storage, bigquery
import pandas as pd
from pyspark.sql import SparkSession
import datetime
import json
import logging


#initialisation de GCS et BQ
GCS = storage.Client()
BQ = bigquery.Client()

#initialisation de la session spark

spark = SparkSession.builder.appName("HospitalAMySQLToLanding").getOrCreate()

# Google Cloud Storage (GCS) Configuration

GCS_BUCKET = "healthcar032025"
HOPITAL_NAME = "hopital-b"
LANDING_PATH = f"gs://{GCS_BUCKET}/landing/{HOPITAL_NAME}/"
ARCHIVE_PATH = f"gs://{GCS_BUCKET}/landing/{HOPITAL_NAME}/archive/"
CONFIG_FILE_PATH = f"gs://{GCS_BUCKET}/configs/load_config.csv"


MYSQL_CONFIG = {
    "url": "jdbc:mysql://34.28.132.3:3306/hopital-b?useSSL=false&allowPublicKeyRetrieval=true",
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "azzedine",
    "password": "ZainaSalem1967@"
}

#test de connexion
try:
   
    df = spark.read.jdbc(url=MYSQL_CONFIG["url"], table="patients", properties={
                             "user": MYSQL_CONFIG["user"],
                             "password": MYSQL_CONFIG["password"],
                             "driver": MYSQL_CONFIG["driver"]
                         })
    print('leccture')
   
    df.show(5)  
    
    print("Connexion réussie!")
except Exception as e:
     logger.error("Erreur lors de la connexion à MySQL:", exc_info=True)


leccture


+------------+-----------+------+------+-----------+--------------------+------+----------+--------------------+------------+
|          ID|     F_Name|L_Name|M_Name|        SSN|         PhoneNumber|Gender|       DOB|             Address|ModifiedDate|
+------------+-----------+------+------+-----------+--------------------+------+----------+--------------------+------------+
|HOSP1-000001|   Victoria|Gamble|     Q|318-87-5123|          4902994299|  Male|1994-12-24|7912 Arthur Loaf ...|  2021-06-19|
|HOSP1-000002|     Meghan|  West|     F|110-08-3049|  (703)210-5078x2916|Female|2009-04-03|70600 Destiny Gro...|  2024-07-06|
|HOSP1-000003|Christopher| Klein|     X|528-95-5438|001-234-668-8071x032|  Male|1987-11-15|61743 Vickie Tunn...|  2021-02-22|
|HOSP1-000004|      Grace| Young|     T|821-27-2909|        208.792.7266|Female|2015-02-20|1452 Gregory View...|  2021-12-27|
|HOSP1-000005|     Andrew| Smith|     N|535-73-4816|  574-505-3750x66300|  Male|1973-06-12|15378 Thomas Lock...|  2020

In [3]:
#big query confuguration
BQ_PROJECT = "hybrid-ridge-457421-t8"
BQ_AUDIT_TABLE = f"{BQ_PROJECT}.temp_dataset.audit_log" # stocker l'etat de chaque chargement
BQ_LOG_TABLE = f"{BQ_PROJECT}.temp_dataset.pipeline_logs" #stocker les events de l'excustion de la pipline
BQ_TEMP_PATH = f"{GCS_BUCKET}/temp/"  

log_entries = [] 

def log_event(type_event,message,table=None):
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "event_type": type_event,
        "message": message,
        "table": table
    }
    log_entries.append(log_entry)
    print(f"[{log_entry['timestamp']}] {type_event} - {message}") 
    
    
def save_logs_to_gcs():
    """Enregistrer les logs dans GCS sous forme de fichier JSON"""
    log_filename = f"pipeline_log_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.json"
    log_filepath = f"temp/pipeline_logs/{log_filename}"

    json_data = json.dumps(log_entries, indent=4)

    # Sauvegarde sur GCS
    bucket = GCS.bucket(GCS_BUCKET)
    blob = bucket.blob(log_filepath)
    blob.upload_from_string(json_data, content_type="application/json")

    print(f"✅ Logs successfully saved to GCS at gs://{GCS_BUCKET}/{log_filepath}")
    

        
    


In [4]:
def get_lastes_watemark(table_name):
    query = f"""
        SELECT MAX(load_timestamp) AS latest_timestamp
        FROM `{BQ_AUDIT_TABLE}`
        WHERE tablename = '{table_name}' and data_source = "hopital_b_db"
    """
    query_job = BQ.query(query)
    result = query_job.result()
    for row in result:
        if row.latest_timestamp:
            return row.timestamp
        else:
            return "1900-01-01 00:00:00"
        
    return "1900-01-01 00:00:00"
    

In [5]:
def extract_and_save_in_gcs(table_name,load_type,watermark_col):
   
    
    try:
        if load_type.lower()=='incremental':
            last_watermark=get_lastes_watemark(table_name)
            query=f"""
            (SELECT * FROM {table_name} WHERE {watermark_col} > '{last_watermark}') AS t
            """
        else:
            query=f"(SELECT * FROM {table_name}) AS t"
        
        df = (spark.read.format("jdbc")
                .option("url", MYSQL_CONFIG["url"])
                .option("user", MYSQL_CONFIG["user"])
                .option("password", MYSQL_CONFIG["password"])
                .option("driver", MYSQL_CONFIG["driver"])
                .option("dbtable", query)
                .load())
        log_event("SUCCESS", f" extract des donnees  reussie de {table}", table=table)
        
        today = datetime.datetime.today().strftime('%d%m%Y')
        JSON_FILE_PATH = f"landing/{HOPITAL_NAME}/{table}/{table}_{today}.json"
        
        bucket = GCS.bucket(GCS_BUCKET)
        blob = bucket.blob(JSON_FILE_PATH)
        blob.upload_from_string(df.toPandas().to_json(orient="records", lines=True), content_type="application/json")
        
        log_event("SUCCÈS", f" Fichier JSON enregistré avec succès dans gs://{GCS_BUCKET}/{JSON_FILE_PATH}", table=table)
        
        #table_audit
        audit_df = spark.createDataFrame([
            ("hospital_a_db", table, load_type, df.count(), datetime.datetime.now(), "SUCCESS")], 
            ["data_source", "tablename", "load_type", "record_count", "load_timestamp", "status"])

        (audit_df.write.format("bigquery")
            .option("table", BQ_AUDIT_TABLE)
            .option("temporaryGcsBucket", GCS_BUCKET)
            .mode("append")
            .save())
        
    except Exception as e:
        log_event("ERREUR", f"Erreur lors du traitement de la table {table} : {str(e)}", table=table)

        

In [6]:
def extract_and_save_in_gcs(table_name,load_type,watermark_col):
   
    
    try:
        if load_type.lower()=='incremental':
            last_watermark=get_lastes_watemark(table_name)
            query=f"""
            (SELECT * FROM {table_name} WHERE {watermark_col} > '{last_watermark}') AS t
            """
        else:
            query=f"(SELECT * FROM {table_name}) AS t"
        
        df = (spark.read.format("jdbc")
                .option("url", MYSQL_CONFIG["url"])
                .option("user", MYSQL_CONFIG["user"])
                .option("password", MYSQL_CONFIG["password"])
                .option("driver", MYSQL_CONFIG["driver"])
                .option("dbtable", query)
                .load())
        log_event("SUCCESS", f" extract des donnees  reussie de {table}", table=table)
        
        today = datetime.datetime.today().strftime('%d%m%Y')
        JSON_FILE_PATH = f"landing/{HOPITAL_NAME}/{table}/{table}_{today}.json"
        
        bucket = GCS.bucket(GCS_BUCKET)
        blob = bucket.blob(JSON_FILE_PATH)
        blob.upload_from_string(df.toPandas().to_json(orient="records", lines=True), content_type="application/json")
        
        log_event("SUCCÈS", f" Fichier JSON enregistré avec succès dans gs://{GCS_BUCKET}/{JSON_FILE_PATH}", table=table)
        
        #table_audit
        audit_df = spark.createDataFrame([
            ("hospital_a_db", table, load_type, df.count(), datetime.datetime.now(), "SUCCESS")], 
            ["data_source", "tablename", "load_type", "record_count", "load_timestamp", "status"])

        (audit_df.write.format("bigquery")
            .option("table", BQ_AUDIT_TABLE)
            .option("temporaryGcsBucket", GCS_BUCKET)
            .mode("append")
            .save())
        
    except Exception as e:
        log_event("ERREUR", f"Erreur lors du traitement de la table {table} : {str(e)}", table=table)

        

In [8]:
def move_existing_files_to_archive(table):
    blobs = list(GCS.bucket(GCS_BUCKET).list_blobs(prefix=f"landing/{HOPITAL_NAME}/{table}/"))
    existing_files = [blob.name for blob in blobs if blob.name.endswith(".json")]
    
    if not existing_files:
        log_event("INFO", f"Aucun fichier existant pour la table {table}")
        return
    
    for file in existing_files:
        source_blob = GCS.bucket(GCS_BUCKET).blob(file)

        # Extract Date from File Name
        date_part = file.split("_")[-1].split(".")[0]
        year, month, day = date_part[-4:], date_part[2:4], date_part[:2]

        # Move to Archive
        archive_path = f"landing/{HOPITAL_NAME}/archive/{table}/{year}/{month}/{day}/{file.split('/')[-1]}"
        destination_blob = GCS.bucket(GCS_BUCKET).blob(archive_path)

        # Copy file to archive and delete original
        GCS.bucket(GCS_BUCKET).copy_blob(source_blob, GCS.bucket(GCS_BUCKET), destination_blob.name)
        source_blob.delete()

        log_event("INFO", f"Fichier {file} déplacé vers {archive_path}", table=table)
    
    
    
    
    

In [9]:
def read_config_file():
    df = spark.read.csv(CONFIG_FILE_PATH, header=True)
    log_event("INFO", "✅ Fichier de configuration lu avec succès")
    return df

# read config file
config_df = read_config_file()

for row in config_df.collect():
    if row["is_active"] == '1' and row["datasource"] == "hospital_b_db": 
        db, src, table, load_type, watermark, _, targetpath = row
        move_existing_files_to_archive(table)
        extract_and_save_in_gcs(table, load_type, watermark)
        
save_logs_to_gcs()


[2025-05-04T17:38:39.940829] INFO - ✅ Fichier de configuration lu avec succès
[2025-05-04T17:38:40.304421] INFO - Aucun fichier existant pour la table encounters
[2025-05-04T17:38:41.539800] SUCCESS -  extract des donnees  reussie de encounters


[2025-05-04T17:38:43.080956] SUCCÈS -  Fichier JSON enregistré avec succès dans gs://healthcar032025/landing/hopital-b/encounters/encounters_04052025.json


[2025-05-04T17:38:58.560719] INFO - Aucun fichier existant pour la table patients
[2025-05-04T17:38:59.573377] SUCCESS -  extract des donnees  reussie de patients
[2025-05-04T17:39:00.518748] SUCCÈS -  Fichier JSON enregistré avec succès dans gs://healthcar032025/landing/hopital-b/patients/patients_04052025.json


[2025-05-04T17:39:13.146882] INFO - Aucun fichier existant pour la table transactions
[2025-05-04T17:39:14.151096] SUCCESS -  extract des donnees  reussie de transactions


[2025-05-04T17:39:17.408465] SUCCÈS -  Fichier JSON enregistré avec succès dans gs://healthcar032025/landing/hopital-b/transactions/transactions_04052025.json


[2025-05-04T17:39:24.005249] INFO - Aucun fichier existant pour la table providers
[2025-05-04T17:39:24.260863] SUCCESS -  extract des donnees  reussie de providers
[2025-05-04T17:39:24.699963] SUCCÈS -  Fichier JSON enregistré avec succès dans gs://healthcar032025/landing/hopital-b/providers/providers_04052025.json


[2025-05-04T17:39:31.119220] INFO - Aucun fichier existant pour la table departments
[2025-05-04T17:39:31.362525] SUCCESS -  extract des donnees  reussie de departments
[2025-05-04T17:39:31.841223] SUCCÈS -  Fichier JSON enregistré avec succès dans gs://healthcar032025/landing/hopital-b/departments/departments_04052025.json


✅ Logs successfully saved to GCS at gs://healthcar032025/temp/pipeline_logs/pipeline_log_20250504174009.json
